## PMP solved by Genetic Algorithm (GA)

In [1]:
from GA import GeneticAlgorithm
import random
import numpy as np
from matplotlib import pyplot as plt

## Load the real-world datasets

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### LandScan Population Distribution
LandScan data is preprocessed and excluded the regions with no night-time population. Each point in this dataset represents the population aggregated to the centroid of the corresponding grid cell.

In [3]:
%%time
ls = gpd.read_file("./real/haidian_community_pro.shp")
ls['POINT_X'] = ls.geometry.x
ls['POINT_Y'] = ls.geometry.y
ls.head(3)

Wall time: 174 ms


,经度84,纬度84,POINT_X,POINT_Y,NEAR_FID,NEAR_DIST,NEAR_X,NEAR_Y,geometry
0,116.268247,39.970932,950178.402354,4.439620e+06,53,822.227299,950742.699785,4.439022e+06,POINT (950178.402 4439619.552)
1,116.278045,39.965943,951048.965144,4.439115e+06,53,320.168328,950742.699785,4.439022e+06,POINT (951048.965 4439114.857)
2,116.216655,39.996868,945598.585992,4.442241e+06,45,2297.119273,947792.780208,4.442921e+06,POINT (945598.586 4442241.439)


In [4]:
sitedf = gpd.read_file("./real/haidian_hospital_pro.shp")
sitedf['POINT_X'] = sitedf.geometry.x
sitedf['POINT_Y'] = sitedf.geometry.y
# sites = np.array(sitedf[['NORM_X', 'NORM_Y']], dtype=np.float64)
# print("The number of billboards in Seattle area is ", len(sitedf))
sitedf.head(3)

,经度84,纬度84,POINT_X,POINT_Y,geometry
0,116.257490,40.023362,948914.176201,4.445391e+06,POINT (948914.176 4445390.949)
1,116.287560,39.993906,951677.729470,4.442270e+06,POINT (951677.729 4442270.340)
2,116.353798,39.981356,957422.791546,4.441215e+06,POINT (957422.792 4441214.652)


## Normalization

In [5]:
def Normalization(x, y):
    max_x, max_y = np.max(x), np.max(y)
    min_x, min_y = np.min(x), np.min(y)
    S_x = (max_x-min_x)
    S_y = (max_y-min_y)
    S = max(S_x, S_y)
    new_x, new_y = (x-min_x)/S, (y-min_y)/S
    data_xy = np.vstack((new_x, new_y))
    Data = data_xy.T
    return new_x, new_y, S

In [6]:
ls_X = np.array(ls['POINT_X'])
ls_Y = np.array(ls['POINT_Y'])
bbs_X = np.array(sitedf['POINT_X'])
bbs_Y = np.array(sitedf['POINT_Y'])
X = np.concatenate([ls_X, bbs_X])
Y = np.concatenate([ls_Y, bbs_Y])
NORM_X, NORM_Y, S = Normalization(X, Y)
ls['NORM_X'] = NORM_X[:len(ls)]
ls['NORM_Y'] = NORM_Y[:len(ls)]
sitedf['NORM_X'] = NORM_X[len(ls):]
sitedf['NORM_Y'] = NORM_Y[len(ls):]

In [7]:
users = np.array(ls[['NORM_X', 'NORM_Y']])
facilities = np.array(sitedf[['NORM_X', 'NORM_Y']])
p = 15
dist = np.sum((users[:, np.newaxis, :] - facilities[np.newaxis, :, :]) ** 2, axis=-1) ** 0.5

In [10]:
genetic = GeneticAlgorithm(len(ls), len(sitedf), p, dist)
genetic.optimize()
obj = genetic.top_chromosome.fitness
centers = genetic.top_chromosome.content
time = genetic.time

print("The Set of centers are: %s" % centers)
print("The objective is: %s" % obj)


Current top solution: [30, 1, 35, 8, 18, 47, 5, 12, 77, 23, 56, 70, 75, 26, 63] f=182
Hypermutation...
Hypermutation...
Hypermutation...

Final top solution: [50, 66, 73, 47, 20, 19, 61, 24, 39, 43, 55, 18, 0, 7, 26] f=153
Time: 00:00:44.8655
The Set of centers are: [50, 66, 73, 47, 20, 19, 61, 24, 39, 43, 55, 18, 0, 7, 26]
The objective is: 153.55013894308723
